In [1]:
import pickle
import pandas as pd

In [2]:
with open('../../week_1/models/regressor.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)


In [3]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')


In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [6]:
# Question 1 -What's the mean predicted duration for this dataset?
print(f'y_pred mean is {y_pred.mean()}')

y_pred mean is 25.81928513703315


In [8]:
# Question 2 - What's the size of the output file?
year = 2021
month = 2
taxi_type = 'fhv'

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

# create an empty dataframe for the prediction and ride_id
df_result = pd.DataFrame()
df_result["predictions"] = y_pred
df_result['ride_id'] = df['ride_id']

print(df_result.info())

# output file s3 bucket to store the dataframe df_result
output_file = f's3://nyc-duration-prediction-usman/taxi_type={taxi_type}/year={year:04d}/month={month:02d}.parquet'

# storing the dataframe as parquet file in s3 bucket
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
print('Save df_result into {output_file} in the s3 bucket}')

Future exception was never retrieved
future: <Future finished exception=ClientOSError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)>
aiohttp.client_exceptions.ClientOSError: [WinError 10053] An established connection was aborted by the software in your host machine


Save df_result into {output_file} in the s3 bucket}


In [9]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990113 entries, 0 to 990112
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   predictions  990113 non-null  float64
 1   ride_id      943966 non-null  object 
dtypes: float64(1), object(1)
memory usage: 15.1+ MB
